In [16]:
library(lme4)
library(lmerTest)
library(stargazer)
library(MatchIt)
library("lmtest") #coeftest
library("sandwich") #vcovCL

Loading required package: Matrix

Warning message:
“package ‘lmerTest’ was built under R version 3.6.3”

Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step


Warning message:
“package ‘stargazer’ was built under R version 3.6.3”

Please cite as: 


 Hlavac, Marek (2018). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.2. https://CRAN.R-project.org/package=stargazer 


Warning message:
“package ‘lmtest’ was built under R version 3.6.3”
Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Warning message:
“package ‘sandwich’ was built under R version 3.6.3”


In [17]:
country.pair.dirs <- list.dirs(path="/shared/2/projects/cross-lingual-exchange/data/betweenness_dataframes")
country.pair.dirs <- country.pair.dirs[2:length(country.pair.dirs)]
out.file <- "/shared/2/projects/cross-lingual-exchange/results/overall_att_betweenness.tsv"

In [12]:
x <- data.frame('Estimate', 'Std. Error', 'P-value')
write.table(x, file = out.file, sep = "\t", append = TRUE, quote = FALSE,
            col.names = FALSE, row.names = FALSE)

In [18]:
all.data.frames <- vector(mode="list",length=2*length(country.pair.dirs))

In [19]:
for(i in 1:length(country.pair.dirs)){
    country.pair.dir <- country.pair.dirs[[i]]
    country.pair <- unlist(strsplit(basename(country.pair.dir),'_'))
    filename1 <- paste(country.pair.dir,"/",country.pair[[1]],".tsv",sep='')
    filename2 <- paste(country.pair.dir,"/",country.pair[[2]],".tsv",sep='')
    all.data.frames[[(2*i)-1]] <- read.table(filename1,header=TRUE,sep="\t")
    all.data.frames[[(2*i)]] <- read.table(filename2,header=TRUE,sep="\t")
}

In [25]:
country.pair[[1]]

[1] "AM"

In [39]:
i

[1] 232

In [20]:
df <- do.call("rbind", all.data.frames)

In [41]:
head(df)

id_str,country,monolingual_majority,monolingual_minority,is_bilingual,num_neighbors,num_neighbors_monolingual,num_neighbors_bilingual,has_monolingual_neighbor,has_bilingual_neighbor,...,betw,pagerank,verified,friends_count,followers_count,favourites_count,statuses_count,account_age,post_rate,decahose_tweet_count
7.623328e+17,AM,0,1,0,6,5,0,1,0,...,2.442811e-07,1.195114e-07,0,577,1378,6546,4521,1374,3.2903930,222
1.649689e+09,AM,0,1,0,19,16,2,1,1,...,4.050710e-06,3.480645e-07,0,1873,4087,13748,2911,2495,1.1667335,331
3.904403e+08,AM,0,1,0,1,1,0,1,0,...,0.000000e+00,3.512426e-08,0,585,176,8143,11692,3156,3.7046895,292
1.563036e+07,AM,0,1,0,7,7,0,1,0,...,7.237188e-08,1.084794e-07,0,1845,1311,34115,67106,4328,15.5050832,822
2.561019e+09,AM,0,1,0,7,4,0,1,0,...,6.357928e-08,9.122556e-08,0,84,4241,44291,59919,2204,27.1864791,1269
3.148786e+09,AM,0,1,0,1,1,0,1,0,...,0.000000e+00,4.974627e-08,0,612,1839,807,1587,1852,0.8569114,15


In [42]:
df$id_str <- NULL
df$country <- NULL 
df <- df[!rowSums(df < 0), ]
df['log_degree'] <- log(df$num_neighbors + 1)
df['log_friends'] <- log(df$friends_count + 1)
df['log_followers'] <- log(df$followers_count + 1)
df['log_statuses'] <- log(df$statuses_count + 1)
df['log_tweets'] <- log(df$decahose_tweet_count + 1)
df['log_favourites'] <- log(df$favourites_count + 1)
df['scaled_betw'] <- 1000000 * df$betw
df['log_betw'] <- log(df$scaled_betw + 1)
df['log_rate'] <- log(df$post_rate + 1)
df['log_bilingual_neighbors'] <- log(df$num_neighbors_bilingual + 1)
df['log_age'] <- log(df$account_age + 1) # make that linear

In [43]:
m <- matchit(is_bilingual ~ log_degree 
            + verified + log_friends + log_followers + log_statuses + log_favourites
            + log_age + log_rate + log_tweets,
            data=df,method='subclass',subclass=min(sum(df$is_bilingual),25))

In [44]:
m.data <- match.data(m)

In [45]:
lm.out <- lm(log_betw ~ is_bilingual + log_degree 
                     + verified + log_friends + log_followers + log_statuses + log_favourites
                     + log_age + log_rate + log_tweets
                     ,data=m.data,weights=weights)

In [49]:
res <- coeftest(lm.out,vcov.=vcovHC)
x <- c(res[2,1],res[2,2],res[2,4])
x <- as.matrix(t(x))
write.table(x, file = out.file, sep = "\t", append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

In [50]:
x

0.03362419,0.000483029,0


In [48]:
res

ERROR: Error in eval(expr, envir, enclos): object 'res' not found


In [54]:
avg_log_betw <- mean(df[['log_betw']])

In [55]:
att <- res[2,1]

In [56]:
avg_log_betw

[1] 0.4131538

In [57]:
(avg_log_betw + att) / avg_log_betw

[1] 1.081384

In [58]:
att / avg_log_betw

[1] 0.08138419

In [59]:
exp(att) / exp(avg_log_betw)

[1] 0.6841832

In [61]:
exp(att)

[1] 1.034196

In [62]:
exp(avg_log_betw)

[1] 1.511577

In [64]:
exp(att) / mean(df[['scaled_betw']])

[1] 0.2992804

In [5]:
x

ERROR: Error in eval(expr, envir, enclos): object 'x' not found


In [6]:
avg_log_betw <- 0.413153787720591
att <- 0.03362419

In [7]:
att / avg_log_betw #8 percent increase in log betweenness

[1] 0.0813842

In [8]:
(exp(att + avg_log_betw) - 1)/(exp(avg_log_betw)-1)  #10 percent increase in "normal" betweenness

[1] 1.10104

In [13]:
(exp(att + avg_log_betw) )/(avg_scaled_betw)

[1] 0.4523854

In [10]:
avg_scaled_betw <- exp(att)/0.299280358063564

In [11]:
avg_scaled_betw

[1] 3.455609

In [15]:
exp(att)

[1] 1.034196

In [25]:
nrow(df)

[1] 167845570

In [26]:
colnames(df)

[1] "id_str"                      "country"                    
 [3] "monolingual_majority"        "monolingual_minority"       
 [5] "is_bilingual"                "num_neighbors"              
 [7] "num_neighbors_monolingual"   "num_neighbors_bilingual"    
 [9] "has_monolingual_neighbor"    "has_bilingual_neighbor"     
[11] "num_neighbors_same_country"  "num_neighbors_other_country"
[13] "has_neighbors_same_country"  "has_neighbors_other_country"
[15] "betw"                        "pagerank"                   
[17] "verified"                    "friends_count"              
[19] "followers_count"             "favourites_count"           
[21] "statuses_count"              "account_age"                
[23] "post_rate"                   "decahose_tweet_count"

In [33]:
df['scaled_betw'] <- 1000000 * df$betw
df['log_betw'] <- log(df$scaled_betw + 1)
treated_scaled_betw <- mean(df[df['is_bilingual']==1,][['scaled_betw']])
treated_log_betw <- mean(df[df['is_bilingual']==1,][['log_betw']])

In [37]:
att  # increase of 0.0334 in log betweenness centrality

[1] 0.03362419

In [36]:
att / (treated_log_betw-att)   # 9.349 percent increase in log betweenness centrality

[1] 0.0934921

In [39]:
t0 <- (treated_log_betw + 1 - exp(att))/exp(att)
(treated_log_betw - t0)/t0    # 13.269 percent increase in betweenness centrality

[1] 0.1326855

In [40]:
unique_users <- length(unique(df[['id_str']]))

In [41]:
unique_users   # 7.73 million unique users

[1] 7730929

In [43]:
head(all.data.frames[3])

id_str,country,monolingual_majority,monolingual_minority,is_bilingual,num_neighbors,num_neighbors_monolingual,num_neighbors_bilingual,has_monolingual_neighbor,has_bilingual_neighbor,...,betw,pagerank,verified,friends_count,followers_count,favourites_count,statuses_count,account_age,post_rate,decahose_tweet_count
7.623328e+17,AM,0,1,0,5,4,0,1,0,...,3.112968e-03,2.520727e-06,0,577,1378,6546,4521,1374,3.29039301,222
1.649689e+09,AM,0,1,0,18,15,2,1,1,...,2.548003e-03,7.856198e-06,0,1873,4087,13748,2911,2495,1.16673347,331
3.904403e+08,AM,0,1,0,1,1,0,1,0,...,0.000000e+00,8.017638e-07,0,585,176,8143,11692,3156,3.70468948,292
1.563036e+07,AM,0,1,0,5,5,0,1,0,...,1.324138e-04,2.284742e-06,0,1845,1311,34115,67106,4328,15.50508318,822
3.148786e+09,AM,0,1,0,1,1,0,1,0,...,0.000000e+00,1.064523e-06,0,612,1839,807,1587,1852,0.85691145,15
2.733022e+08,AM,0,1,0,15,7,1,1,1,...,3.008700e-05,1.219911e-05,0,943,11649,2384,11306,3333,3.39213921,270
1.010487e+18,AM,0,1,0,1,0,0,0,0,...,0.000000e+00,2.488268e-06,0,734,247,6547,2437,649,3.75500770,60
9.113593e+17,AM,1,0,0,25,21,4,1,1,...,7.368642e-07,4.209939e-06,0,78,81,38692,49,910,0.05384615,78
7.194970e+17,AM,0,0,1,91,63,19,1,1,...,6.727183e-05,1.570788e-05,0,337,769,33789,21788,1505,14.47707641,662
8.864872e+17,AM,1,0,0,46,35,9,1,1,...,1.398026e-06,7.621996e-06,0,263,199,17570,3734,764,4.88743456,350


In [56]:
network_subset_sizes <- rep(0,length(country.pair.dirs))

In [57]:
for(i in 1:length(network_subset_sizes)){
    network_subset_sizes[[i]] <- nrow(all.data.frames[[2*i-1]]) + nrow(all.data.frames[[2*i]])
}

In [58]:
network_subset_sizes

[1] 2362427  117658  314700   18950  889122   25040  818327 2388751   25335
 [10]  129740  166076   99822   12233   84163  159442  320434   44139 1165874
 [19]   18049  131828  886668   22724  818035 2369250  121405  163616   97267
 [28]   81806  157135  318603   41802   15697 2363408  118458  317168   27876
 [37]  139968  897164  825567 2380043  134203   19851   93226 1172025  138047
 [46]  896048  824340 2382466  136203  173067   91435  328571   51245   15411
 [55] 1170752  132117  125753 1009348  144126  939637 2623226  144439  126952
 [64]  351642  285465  125074  129350  220008  131417  203673  278816  440311
 [73]  163398 1288662  889896  818398 2378873  131592  166966  100129   16987
 [82]   85202   45663 1164964  884191   20466  812695 2367178  119079  160782
 [91]   93696   78939  154387  315466   38965 1159652  902125 1706354 3455993
[100]  902274  884977 1190028  894877  884474 1052987  883250  887200  976361
[109]  889395  961501 1071270  883252 1197474  921195  884824 2043857  830361
[118] 2393700  147475  179186  112051  172915  333693   57807 1177020 3427115
[127]  830464  813384 1162690  823250  812741  976742  811285  815558  904909
[136]  817546  889565  968666  811782 1125729  849453  813152 1973409 2391971
[145] 2368433 2376768 2364895 2610956 2363108 2369416 2563975 2378713 2474167
[154] 2592565 2364947 2717123 2428798 2365723 3608664  145461  179469 1177589
[163]  122431  162021   94903   80237  155704   40229 1160209  131762  119612
[172]  356337  117605  124267  299584  131503  222545  340406  119030  470485
[181]  180218  120707 1357653  326749   94570   39963 1159457  160128  164410
[190]  252903  238225  315894  160486  474536  198147  162165 1319662    4702
[199]   92959   78326  153783   38302 1157914   97271   82633  319861 1162447
[208]   99365  171647  246603   93142  407393  131082 1252545   84574  160030
[217]   45548 1164239  231829  393392  116346   80386 1236080  153949  468051
[226]  191820 1312607 1159187  355757 1472673 1196308 1159779

In [59]:
sum(network_subset_sizes)

[1] 167845570

In [60]:
summary(network_subset_sizes)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   4702  126652  319232  723472 1057558 3608664 

In [62]:
country.pair.dirs[[which(network_subset_sizes==max(network_subset_sizes))]]

[1] "/shared/2/projects/cross-lingual-exchange/data/betweenness_dataframes/GB_TR"

In [63]:
country.pair.dirs[[which(network_subset_sizes==min(network_subset_sizes))]]

[1] "/shared/2/projects/cross-lingual-exchange/data/betweenness_dataframes/LT_LV"

In [64]:
length(network_subset_sizes)

[1] 232

In [66]:
network_subset_sizes

[1] 2362427  117658  314700   18950  889122   25040  818327 2388751   25335
 [10]  129740  166076   99822   12233   84163  159442  320434   44139 1165874
 [19]   18049  131828  886668   22724  818035 2369250  121405  163616   97267
 [28]   81806  157135  318603   41802   15697 2363408  118458  317168   27876
 [37]  139968  897164  825567 2380043  134203   19851   93226 1172025  138047
 [46]  896048  824340 2382466  136203  173067   91435  328571   51245   15411
 [55] 1170752  132117  125753 1009348  144126  939637 2623226  144439  126952
 [64]  351642  285465  125074  129350  220008  131417  203673  278816  440311
 [73]  163398 1288662  889896  818398 2378873  131592  166966  100129   16987
 [82]   85202   45663 1164964  884191   20466  812695 2367178  119079  160782
 [91]   93696   78939  154387  315466   38965 1159652  902125 1706354 3455993
[100]  902274  884977 1190028  894877  884474 1052987  883250  887200  976361
[109]  889395  961501 1071270  883252 1197474  921195  884824 2043857  830361
[118] 2393700  147475  179186  112051  172915  333693   57807 1177020 3427115
[127]  830464  813384 1162690  823250  812741  976742  811285  815558  904909
[136]  817546  889565  968666  811782 1125729  849453  813152 1973409 2391971
[145] 2368433 2376768 2364895 2610956 2363108 2369416 2563975 2378713 2474167
[154] 2592565 2364947 2717123 2428798 2365723 3608664  145461  179469 1177589
[163]  122431  162021   94903   80237  155704   40229 1160209  131762  119612
[172]  356337  117605  124267  299584  131503  222545  340406  119030  470485
[181]  180218  120707 1357653  326749   94570   39963 1159457  160128  164410
[190]  252903  238225  315894  160486  474536  198147  162165 1319662    4702
[199]   92959   78326  153783   38302 1157914   97271   82633  319861 1162447
[208]   99365  171647  246603   93142  407393  131082 1252545   84574  160030
[217]   45548 1164239  231829  393392  116346   80386 1236080  153949  468051
[226]  191820 1312607 1159187  355757 1472673 1196308 1159779

In [74]:
all.countries <- list()
for(country.pair.dir in country.pair.dirs){
    country.pair <- unlist(strsplit(basename(country.pair.dir),'_'))
    all.countries <- append(all.countries,country.pair[[1]]) 
    all.countries <- append(all.countries,country.pair[[2]]) 
    }

In [75]:
length(all.countries)

[1] 464

In [77]:
unique(all.countries)

[[1]]
[1] "AM"

[[2]]
[1] "GB"

[[3]]
[1] "IE"

[[4]]
[1] "RU"

[[5]]
[1] "AT"

[[6]]
[1] "CZ"

[[7]]
[1] "ES"

[[8]]
[1] "FI"

[[9]]
[1] "FR"

[[10]]
[1] "GR"

[[11]]
[1] "IT"

[[12]]
[1] "NL"

[[13]]
[1] "NO"

[[14]]
[1] "PL"

[[15]]
[1] "PT"

[[16]]
[1] "SE"

[[17]]
[1] "TR"

[[18]]
[1] "AZ"

[[19]]
[1] "BY"

[[20]]
[1] "DE"

[[21]]
[1] "BG"

[[22]]
[1] "LV"

[[23]]
[1] "SI"

[[24]]
[1] "DK"

[[25]]
[1] "EE"

[[26]]
[1] "HU"

[[27]]
[1] "LT"

[[28]]
[1] "IL"

[[29]]
[1] "IS"

[[30]]
[1] "RO"

In [79]:
nrow(df)

[1] 167845570

In [82]:
head(df['id_str'])

id_str
7.623328e+17
1.649689e+09
3.904403e+08
1.563036e+07
2.561019e+09
3.148786e+09


In [ ]:
as.character(df['id_str'])